# ADHD Medication Data ETL Pipeline

**Objective:** Create an automated ETL (Extract, Transform, Load) pipeline that retrieves public health data from the Swedish National Board of Health and Welfare (Socialstyrelsen) API and stores it in a SQL database for future analysis projects.

**Scheduling:** The pipeline will be scheduled to run annually at the beginning of each year (starting January 2026) to capture the complete previous year's data when it becomes available.

## Background

### ADHD Medications in Sweden

Sweden has five approved medication substances for ADHD treatment, categorized into two groups:

**Central Stimulants:**
- Methylphenidate (e.g., Concerta, Ritalin) - N06BA04
- Lisdexamfetamine (Elvanse) - N06BA12  
- Dexamfetamine (Attentin) - N06BA02

**Non-Central Stimulants:**
- Atomoxetine (Strattera) - N06BA09
- Guanfacine (Intuniv) - C02AC02

### ATC Codes Used
- **N06BA**: Centrally acting sympathomimetics (excluding N06BA07 Modafinil)
- **C02AC02**: Guanfacine
- **N06BA (excluding Modafinil and Atomoxetine)**: For specific stimulant analysis

### Key Metric: Patients/1000 Inhabitants

**Definition:** Number of patients divided by the total population in the relevant group (age group, region, etc.) and multiplied by 1000.

**Why This Metric:**
- **Prevalence measure**: Shows how many are treated for ADHD
- **Comparable over time**: Normalized per population
- **Stable measure**: Less affected by administrative changes  
- **Interpretable**: "How common is ADHD treatment?"
- **Best for trend analysis**: Accounts for population size differences

*Note: Population data uses January 1st of the respective year. Due to how population and patient age are defined, patient numbers may occasionally exceed population numbers for certain selections.*

### Data Sources
- [Vård och Insats - ADHD Treatment](https://www.vardochinsats.se/adhd/behandling-och-stoed/laekemedelsbehandling/)
- [Swedish Medical Products Agency - ADHD Treatment Recommendations](https://www.lakemedelsverket.se/sv/behandling-och-forskrivning/behandlingsrekommendationer/sok-behandlingsrekommendationer/lakemedel-vid-adhd--behandlingsrekommendation)

---

## ETL Pipeline Overview

This notebook will:
1. **Extract**: Fetch ADHD medication data from Socialstyrelsen API
2. **Transform**: Clean and structure the data for analysis
3. **Load**: Store the processed data in a SQL database

The resulting database will serve as the foundation for future trend analysis and forecasting projects.

In [2]:
import csv
import json
from typing import Dict, List, Optional
import requests
import re
import itertools

## Fetch data on ADHD medication from Socialstyrelsens API

In [5]:
# Base API URL
BASE_RESULT_URL = (
    "https://sdb.socialstyrelsen.se/api/v1/sv/lakemedel/resultat/matt/2" 
)  # Measurement ID 2 = patients/1000 inhabitants

# ATC-codes for ADHD medication
ATC_CODES = {
    "C02AC02": "Guanfacin",
    "N06BA12": "Lisdexamfetamin",
    "N06BA09": "Atomoxetin",
    "N06BA04": "Metylfenidat",
    "N06BA02": "Dexamfetamin"
}

# Default filter values
DEFAULT_REGIONS = list(range(0, 26))     # All counties (0 = Riket, 1-25 = län)
DEFAULT_AGE_GROUPS = [1, 2, 3, 4]        # Age groups (0-4, 5-9, 10-14, 15-19)
DEFAULT_GENDERS = [1, 2, 3]              # Boys, Girls and Both gender
DEFAULT_YEARS = list(range(2006, 2025))  # Years 2006–2024 

# Helper functions for fetch_adhd_medication_data
def _build_api_url(
    atc_codes: str, 
    regions: List[int], 
    age_groups: List[int],
    genders: List[int], 
    years: List[int]
) -> str:
    """
    Build the API URL with specified filters.

    Args:
        atc_codes: ATC code for the medication
        regions: List of region IDs
        age_groups: List of group IDs
        genders: List of gender IDs
        years: List of years

    Returns:
        Complete API URL string
    """
    region_str = ','.join(map(str, regions))
    age_str = ','.join(map(str, age_groups))
    gender_str = ','.join(map(str, genders))
    years_str = ','.join(map(str, years))

    return (
        f"{BASE_RESULT_URL}/atc/{atc_codes}/region/{region_str}/"
        f"alder/{age_str}/kon/{gender_str}/ar/{years_str}"
    )

def _fetch_paginated_data(initial_url: str,
                          headers: Optional[Dict[str, str]] = None) -> List[Dict]:
    """
    Fetch all data from a paginated API endoint.

    Args:
        intial_url: Starting URL for the API request
        headers: Optional HTTP headers
    
    Returns:
        List of all data records from all pages
    
    Raises:
        requests.RequestException: If API request fails
    
    """
    if headers is None:
        headers = {"Accept": "application/json"}
    all_data = []
    url = initial_url

    while url:
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            data_json = response.json()

            # Add data records from current page
            all_data.extend(data_json.get("data", []))

            # Get next page URL
            url = data_json.get("nasta_sida")

        except requests.RequestException as e:
            print(f"Error fetching data from {url}: {e}")

    return all_data

# Main function
def fetch_adhd_medication_data(
    regions: Optional[List[int]] = None,
    age_groups: Optional[List[int]] = None,
    genders: Optional[List[int]] = None,
    years: Optional[List[int]] = None,
    atc_codes: Optional[Dict[str, str]] = None
) -> Dict[str, List[Dict]]:
    """
    Fetch ADHD medication prescription data from Swedish Social Board API.
    
    Args:
        regions: List of region codes (default: all regions 0-25)
        age_groups: List of age group codes (default: [1,2,3,4] for ages 0-19)
        genders: List of gender codes (default: [1,2,3] for men, women, both)
        years: List of years (default: 2006-2025)
        atc_codes: Dict of ATC codes to medication names (default: ADHD medications)
        
    Returns:
        Dictionary with medication names as keys and prescription data as values
        
    Raises:
        requests.RequestException: If API requests fail
    """
    # Use defaults if not provided
    regions = regions or DEFAULT_REGIONS
    age_groups = age_groups or DEFAULT_AGE_GROUPS
    genders = genders or DEFAULT_GENDERS
    years = years or DEFAULT_YEARS
    atc_codes = atc_codes or ATC_CODES
    
    results = {}
    
    for atc_code, medication_name in atc_codes.items():
        print(f"Fetching data for {medication_name} ({atc_code})...")
        
        try:
            url = _build_api_url(atc_code, regions, age_groups, genders, years)
            medication_data = _fetch_paginated_data(url)
            results[medication_name] = medication_data
            
        except requests.RequestException as e:
            print(f"Failed to fetch data for {medication_name}: {e}")
            # Continue with other medications even if one fails
            continue
    
    return results

## Save the fetched data to JSON

In [6]:
def save_results_to_json(
    results: Dict[str, List[Dict]],
    filename: str = "adhd_medication_2006-tbc.json"
) -> None:
    """
    Save the raw data to JSON file.

    Args:
        results: Dictonary of medication data
        filename: Output filename(default: "adhd_medication_2006-tbc.json")
    """
    try:
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
        print(f"Data saved to {filename}")
        
    except IOError as e:
        print(f"Error saving to file {filename}: {e}")
        raise

## Convert JSON data to CSV format

In [7]:
# Mappings from Socialstyrelsens metadata
REGION_MAP = {
    1: "Stockholm",
    3: "Uppsala",
    4: "Södermanland",
    5: "Östergötland",
    6: "Jönköping",
    7: "Kronoberg",
    8: "Kalmar",
    9: "Gotland",
    10: "Blekinge",
    12: "Skåne",
    13: "Halland",
    14: "Västra Götaland",
    17: "Värmland",
    18: "Örebro",
    19: "Västmanland",
    20: "Dalarna",
    21: "Gävleborg",
    22: "Västernorrland",
    23: "Jämtland Härjedalen",
    24: "Västerbotten",
    25: "Norrbotten",
    0: "Riket",
}

KON_MAP = {
    1: "Pojkar",
    2: "Flickor",
    3: "Båda könen",
}

ALDER_MAP = {
    1: "0-4",
    2: "5-9",
    3: "10-14",
    4: "15-19",
}

YEARS = list(range(2006, 2026))  # 2006–2025


def parse_number(s: Optional[str]) -> Optional[float]:
    """Convert Socialstyrelsen values to float (or None)."""
    if s is None:
        return None
    s = re.sub(r"[\u00A0\s]+", "", str(s))
    s = s.replace(",", ".")
    if s == "" or s.lower() in ("na", "n/a", "-", "null"):
        return None
    try:
        return float(s)
    except ValueError:
        return None


def convert_json_to_csv(
    input_json: str = "adhd_lakemedel_2006-tbc.json",
    output_csv: str = "adhd_lakemedel_flat_full.csv",
) -> None:
    """
    Convert ADHD medication data from JSON to flattened CSV.

    Args:
        input_json: Path to input JSON file.
        output_csv: Path to output CSV file.
    """
    with open(input_json, "r", encoding="utf-8") as f:
        data = json.load(f)

    with open(output_csv, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile, delimiter=";")
        writer.writerow(
            ["År", "Läkemedel", "Region", "Kön", "Ålder", "Patienter/1000 invånare"]
        )

        for med_name, records in data.items():
            # Index existing records by (year, region, gender, age)
            record_map = {
                (r["ar"], r["regionId"], r["konId"], r["alderId"]): r
                for r in records
            }

            # Pick one ATC code (same for all records in med_name)
            sample_atc = records[0]["atcId"] if records else ""

            # Loop over all combinations (years × regions × genders × ages)
            for ar, region_id, kon_id, alder_id in itertools.product(
                YEARS, REGION_MAP.keys(), KON_MAP.keys(), ALDER_MAP.keys()
            ):
                r = record_map.get((ar, region_id, kon_id, alder_id), None)
                varde_raw = r.get("varde") if r else None
                patienter_per_1000 = parse_number(varde_raw)

                writer.writerow(
                    [
                        ar,
                        f"{sample_atc} {med_name}",
                        REGION_MAP[region_id],
                        KON_MAP[kon_id],
                        ALDER_MAP[alder_id],
                        "0"
                        if patienter_per_1000 is None
                        else (
                            "{:.3f}".format(patienter_per_1000)
                            .rstrip("0")
                            .rstrip(".")
                        ),
                    ]
                )

    print(f"CSV done: {output_csv}")


## Complete workflow of fetching data ---> save JSON ---> convert to CSV

In [9]:
try:
    # Fetch the ADHD medication data
    results = fetch_adhd_medication_data()
    
    # Save results to JSON file
    save_results_to_json(results)
    
    # Convert JSON to CSV
    convert_json_to_csv(
        input_json="adhd_medication_2006-tbc.json",
        output_csv="adhd_medication_flat_full.csv"
    )
    
    # Print summary
    total_records = sum(len(data) for data in results.values())
    print(f"\nFetched {total_records} total records for {len(results)} medications")

except Exception as e:
    print(f"Error in main execution: {e}")

Fetching data for Guanfacin (C02AC02)...
Fetching data for Lisdexamfetamin (N06BA12)...
Fetching data for Atomoxetin (N06BA09)...
Fetching data for Metylfenidat (N06BA04)...
Fetching data for Dexamfetamin (N06BA02)...
Data saved to adhd_medication_2006-tbc.json
CSV done: adhd_medication_flat_full.csv

Fetched 14806 total records for 5 medications


## Load the data to an pandas dataframe and create a SQLite database for storage

In [11]:
import pandas as pd
import sqlite3

# Run the data fetcher (creates the CSV)
# python adhd_data_fetcher.py

# Load CSV into pandas
df = pd.read_csv("adhd_medication_flat_full.csv", delimiter=";")

# Create SQLite connection and import directly
conn = sqlite3.connect("adhd_data.db")
df.to_sql("adhd_prescriptions", con=conn, if_exists="replace", index=False)
conn.close()

In [12]:
# Connect to the database
conn = sqlite3.connect("adhd_data.db")

# Run a query (example: show first 10 rows)
df_preview = pd.read_sql("SELECT * FROM adhd_prescriptions LIMIT 10;", conn)
print(df_preview)


     År          Läkemedel     Region         Kön  Ålder  \
0  2006  C02AC02 Guanfacin  Stockholm      Pojkar    0-4   
1  2006  C02AC02 Guanfacin  Stockholm      Pojkar    5-9   
2  2006  C02AC02 Guanfacin  Stockholm      Pojkar  10-14   
3  2006  C02AC02 Guanfacin  Stockholm      Pojkar  15-19   
4  2006  C02AC02 Guanfacin  Stockholm     Flickor    0-4   
5  2006  C02AC02 Guanfacin  Stockholm     Flickor    5-9   
6  2006  C02AC02 Guanfacin  Stockholm     Flickor  10-14   
7  2006  C02AC02 Guanfacin  Stockholm     Flickor  15-19   
8  2006  C02AC02 Guanfacin  Stockholm  Båda könen    0-4   
9  2006  C02AC02 Guanfacin  Stockholm  Båda könen    5-9   

   Patienter/1000 invånare  
0                      0.0  
1                      0.0  
2                      0.0  
3                      0.0  
4                      0.0  
5                      0.0  
6                      0.0  
7                      0.0  
8                      0.0  
9                      0.0  


## Create different dataframes for analysis, e.g. extract national data for age group 10-14 from 2020 to 2024.

In [22]:
df_all_medications = pd.read_sql("""
    SELECT
        År AS year,
        Region AS region,
        Kön AS gender,
        Ålder AS age_group,
        Läkemedel as medication,
        "Patienter/1000 invånare" as patients_1000
    FROM adhd_prescriptions
    WHERE Ålder = '10-14' AND Region = 'Riket'
        AND År BETWEEN 2020 AND 2024           
    ORDER BY År, Region, Kön;
""", conn)

print(df_all_medications)

    year region      gender age_group               medication  patients_1000
0   2020  Riket  Båda könen     10-14        C02AC02 Guanfacin           6.76
1   2020  Riket  Båda könen     10-14  N06BA12 Lisdexamfetamin          15.06
2   2020  Riket  Båda könen     10-14       N06BA09 Atomoxetin           6.29
3   2020  Riket  Båda könen     10-14     N06BA04 Metylfenidat          34.40
4   2020  Riket  Båda könen     10-14     N06BA02 Dexamfetamin           0.88
..   ...    ...         ...       ...                      ...            ...
70  2024  Riket      Pojkar     10-14        C02AC02 Guanfacin          19.28
71  2024  Riket      Pojkar     10-14  N06BA12 Lisdexamfetamin          30.49
72  2024  Riket      Pojkar     10-14       N06BA09 Atomoxetin          11.24
73  2024  Riket      Pojkar     10-14     N06BA04 Metylfenidat          66.19
74  2024  Riket      Pojkar     10-14     N06BA02 Dexamfetamin           2.02

[75 rows x 6 columns]
